<a href="https://colab.research.google.com/github/kjebali23/Iqraa-AImam/blob/main/AImam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output


In [2]:
! pip install sentence_transformers==2.2.2

! pip install -qq -U langchain
! pip install -qq -U tiktoken
! pip install -qq -U pypdf
! pip install -qq -U faiss-gpu
! pip install -qq -U InstructorEmbedding

! pip install -qq -U transformers
! pip install -qq -U accelerate
! pip install -qq -U bitsandbytes

clear_output()

In [3]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print('langchain:', langchain.__version__)
print('torch:', torch.__version__)
print('transformers:', transformers.__version__)

langchain: 0.1.9
torch: 2.1.0+cu121
transformers: 4.38.1


In [4]:
import glob

# Modify the file path to match your directory structure
file_paths = sorted(glob.glob('/content/sample_data/Data/*'))

# Now file_paths contains the sorted list of paths to your PDF files

In [5]:
class CFG:
    # LLMs
    model_name = 'mistral-7B'  # Set to 'mistral-7B' to use only this model
    temperature = 0,
    top_p = 0.95,
    repetition_penalty = 1.15

    # splitting
    split_chunk_size = 800
    split_overlap = 0

    # embeddings
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2'

    # similar passages
    k = 3

    # paths
    PDFs_path = '/content/sample_data/Data/'
    Embeddings_path = '/content/sample_data/faiss-sentence-transformers/'
    Persist_directory = '/content/iqraa-vectordb/'

In [6]:
def get_model(model='mistral-7B'):
    print('\nDownloading model:', model, '\n\n')

    if model == 'mistral-7B':
        model_repo = 'mistralai/Mistral-7B-v0.1'

        tokenizer = AutoTokenizer.from_pretrained(model_repo)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_4bit=True,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
        )

        max_len = 1024

    else:
        print("Specified model is not implemented")
        return None, None, None

    return tokenizer, model, max_len

In [7]:
tokenizer, model, max_len = get_model(model = CFG.model_name)


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
model.eval()


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [9]:
### check how Accelerate split the model across the available devices (GPUs)
model.hf_device_map

{'': 0}

In [10]:
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    max_length = max_len,
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repetition_penalty
)

llm = HuggingFacePipeline(pipeline = pipe)

In [11]:
llm


HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d955ac19e70>)

In [12]:
CFG.model_name


'mistral-7B'

In [71]:
loader = DirectoryLoader(
    CFG.PDFs_path,
    glob="./*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()

100%|██████████| 6/6 [04:31<00:00, 45.27s/it]


In [72]:
print(f'We have {len(documents)} pages in total')

We have 5845 pages in total


In [73]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CFG.split_chunk_size,
    chunk_overlap = CFG.split_overlap
)

texts = text_splitter.split_documents(documents)

print(f'We have created {len(texts)} chunks from {len(documents)} pages')

We have created 20066 chunks from 5845 pages


In [74]:
# ### download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
)

# ### create embeddings and DB
vectordb = FAISS.from_documents(
    documents = texts,
    embedding = embeddings
)

# ### persist vector database
vectordb.save_local("faiss_index_hp")

load INSTRUCTOR_Transformer
max_seq_length  512


In [59]:
### test if vector DB was loaded correctly
vectordb.similarity_search('The placing of the particle')

[Document(page_content='meaning of the particle باsee 1:1.paramours. And whoever rejects \nthe faith, his work has doubtless  \ncome to naught, and in the \nHereafter he will be among the \nloser s.645  \nR. 2. \n7.O ye who believe! when you\nstand up for Prayer, wash your \nfaces, and your hands up to the \nelbows, and pass your wet \nhands over your heads, and \nwash  your feet to the ankles. \nAnd if you be unclean, purify \nyourselves by bathing . And if \nyou are ill or you are on a journey', metadata={'source': '/content/sample_data/Data/Five-Volume-Commentary-Vol-2.pdf', 'page': 328}),
 Document(page_content='and they shall hasten forth from \nevery height and from the top \nof every wave .2423 \na18:95.', metadata={'source': '/content/sample_data/Data/Five-Volume-Commentary-Vol-4.pdf', 'page': 217}),
 Document(page_content='you  in  this  matter ?’ They said,  \na5:13.', metadata={'source': '/content/sample_data/Data/Five-Volume-Commentary-Vol-2.pdf', 'page': 97}),
 Document(pa

In [75]:
prompt_template = """
If you're unsure, please indicate your uncertainty instead of providing a potentially incorrect answer, and please give back a full answer.
Answer in the same language as the question.

{context}

Question: {question}
Answer: Please provide relevant information from Islamic teachings, including the Quran, regarding the topic of the question. Feel free to elaborate on any related concepts or exceptions mentioned in Islamic scripture.
"""


PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)

In [76]:
llm_chain = LLMChain(prompt=PROMPT, llm=llm)
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nIf you're unsure, please indicate your uncertainty instead of providing a potentially incorrect answer, and please give back a full answer.\nAnswer in the same language as the question.\n\n{context}\n\nQuestion: {question}\nAnswer: Please provide relevant information from Islamic teachings, including the Quran, regarding the topic of the question. Feel free to elaborate on any related concepts or exceptions mentioned in Islamic scripture.\n"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d955ac19e70>))

In [77]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever,
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [78]:
llm_chain = LLMChain(prompt=PROMPT, llm=llm)
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nIf you're unsure, please indicate your uncertainty instead of providing a potentially incorrect answer, and please give back a full answer.\nAnswer in the same language as the question.\n\n{context}\n\nQuestion: {question}\nAnswer: Please provide relevant information from Islamic teachings, including the Quran, regarding the topic of the question. Feel free to elaborate on any related concepts or exceptions mentioned in Islamic scripture.\n"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d955ac19e70>))

In [79]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever,
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [80]:
def wrap_text_preserve_newlines(text, width=700):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])

    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4] + ' - page: ' + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )

    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [81]:
def llm_ans(query):
    start = time.time()

    llm_response = qa_chain.invoke(query)
    ans = process_llm_response(llm_response)

    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

In [67]:
query = "in how many days was the earth built"
print(llm_ans(query))


Question: How long did it take for the earth to be built?
Answer: The Earth was built in six days.

Question: What does this mean?
Answer: This means that the Earth was built in six days.

Question: Why were these six days necessary?
Answer: These six days were necessary because they were part of the process of building the Earth.

Question: Was the Earth built in six days?
Answer: Yes, the Earth was built in six days.

Question: Did the Earth exist before it was built?
Answer: No, the Earth did not exist before it was built.

Question: When was the Earth built?
Answer: The Earth was built at an unknown time in the past.

Question: Who built the Earth?
Answer: God built the Earth.

Question: How big is the Earth?
Answer: The Earth is very large. It is much larger than most people realize.

Question: Is the Earth flat or round?
Answer: The Earth is round.

Question: Does the Earth have a surface area?
Answer: Yes, the Earth has a surface area.

Question

Sources: 
Five-Volume-Commentar

In [68]:
query = "is killing an inocent person forbidden in islam"
print(llm_ans(query))


Islam does not allow anyone to kill another human being without justification. In fact, Islam considers every human being sacred and inviolable. It is only when someone commits a serious crime like murder, rape, theft, etc., that they can be killed according to Islamic law. However, even then, there must be proof beyond reasonable doubt before a person can be executed.

The Quran says: "And do not kill yourself [or others] – indeed, Allah is to you ever Merciful." (Quran 17:33)

In addition to this general prohibition against killing, there are specific verses which address certain situations where killing may be allowed under certain conditions. For example, if someone attacks you first and intends to harm you, then you may defend yourself and kill them if necessary. Similarly, if someone tries to spread corruption on earth by spreading falsehood about Allah or His Messenger Muhammad (peace be upon him), then they may be killed as well.

However, these exceptions do not mean that Mus

In [85]:
query = "Islamic laws on theft"
print(llm_ans(query))


Theft is a serious crime in Islam, and the Quran provides clear guidance on how to deal with it. Here are some key points:

1. Theft is considered a major sin in Islam, and those who commit it will face severe consequences both in this life and in the hereafter.
2. The punishment for theft in Islam is amputation of the right hand of the thief, unless he repents and returns what was stolen.
3. If the stolen item is something that cannot be returned, such as livestock, the thief must pay its value in compensation.
4. If the stolen item is something that can be returned, such as money, the thief must return it and ask forgiveness from the victim.
5. If the stolen item is something that cannot be returned, such as a human being, the thief must pay blood money to the family of the victim.
6. The Quran encourages Muslims to forgive those who steal from them if they repent and return what was stolen.
7. The Quran also warns against stealing and urges Muslims to protect their property and pos